In [1]:
#!pip install scikit-learn --force
#!pip install catboost

In [2]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import os
import importlib

pd.set_option('display.max.columns', 300)

from core.calculator.storage import ModelDB
from core.calculator.core import ForecastConfig
from core.calculator.core import TrainingManager
from core.calculator.core import ForecastConfig
from core.calculator.core import ForecastEngine

from core.calculator.deposits import DepositsCalculationType
from core.calculator.deposits import DepositIterativeCalculator

from core.definitions import *
from core.project_update import load_portfolio

from core.models import DepositModels
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
# Дата из который мы прогнозируем (пока что не меняется)
train_end = datetime(year=2023, month=9, day=30)

# Горизонт прогноза в месяцах
horizon = 3

In [4]:
train_end

datetime.datetime(2023, 9, 30, 0, 0)

In [5]:
# Данные для прогноза
scenario_data = {
    # Ожидаемый баланс на первую дату прогноза, задавать необязательно
     'expected_amount':      [np.nan for h in range(horizon)],
    # ССВ
     'SSV':                  [0.48 for h in range(horizon)],
    # ФОР
     'FOR':                  [4.5 for h in range(horizon)],
    # Трансфертные ставки
     'VTB_ftp_rate_[90d]':   [12.3 for h in range(horizon)],
     'VTB_ftp_rate_[180d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[365d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[548d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[730d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[1095d]': [12 for h in range(horizon)],
    
    # Маржа бизнеса по срочностям
     'margin_[90d]':         [0.1 for h in range(horizon)],
     'margin_[180d]':        [0.1 for h in range(horizon)],
     'margin_[365d]':        [0.1 for h in range(horizon)],
     'margin_[548d]':        [0.1 for h in range(horizon)],
     'margin_[730d]':        [0.2 for h in range(horizon)],
     'margin_[1095d]':       [0.2 for h in range(horizon)],
    
    # Спред Привилегия - Массовый (на сколько в среднем ставки по сегменту Привилегия больше чем ставки по массовому сегменту)
     'priv_spread':          [0.4 for h in range(horizon)],
    # Спред ВИП - Массовый (на сколько в среднем ставки по сегменту ВИП больше чем ставки по массовому сегменту)
     'vip_spread':           [0.8 for h in range(horizon)],
    
    # Ниже три спреда по разным типам опциональности по отношению к безопциональным вкладам (Подразумевается, что они, как правило, отрицательные)
    # r - возможности пополнения, s - возможность расходных операций
    
    # На сколько ставка по расходным вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r0s1_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s0_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по расходно-пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s1_spread':          [-1.2 for h in range(horizon)],
    
    # Ставка по лучшему предложению сбера
     'SBER_max_rate':        [11.2, 11.2, 11.2],
    
    # Базовая ставка по НС
     'SA_rate':              [5 for h in range(horizon)]
}
scenario_df_user = pd.DataFrame(scenario_data)

In [6]:
# если хотим обучить модели

from core.models.utils import run_spark_session
spark = run_spark_session('check_calc')

#spark = None #если без обучения

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/13 11:59:17 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
23/11/13 11:59:18 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
23/11/13 11:59:19 WARN yarn.Client: Same path resource file:///opt/cloudera/parcels/AnacondaPy37/jars/spark-tree-plotting-0.2.jar added multiple times to distributed cache.
23/11/13 11:59:33 WARN ipc.Client: Exception encountered while connecting to the server : org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyExceptio

In [7]:
folder = '../data/trained_models'

sqlite_filepath = os.path.join(folder, 'modeldb_test.bin')

DB_URL = f"sqlite:///{sqlite_filepath}"
model_db = ModelDB(DB_URL)


In [8]:
ENV_NAME = 'hmelevskoi_env'

os.environ['CC'] = 'x86_64-conda-linux-gnu-gcc'
os.environ['CXX'] = 'x86_64-conda-linux-gnu-g++'
os.environ['PATH'] = os.path.abspath(f'/tmp/envs/{ENV_NAME}/bin') + ':' + os.environ['PATH']

In [9]:
def retrain_on_date(train_end):
    
    scenario_df = preprocess_scenario(scenario_df_user, train_end, horizon)
    
    port_folder = '../data/portfolio_data'
    portfolio = load_portfolio(train_end, port_folder)
    
    

    config: ForecastConfig = ForecastConfig(
        first_train_end_dt = train_end,
        horizon = horizon,
        trainers = DepositModels.trainers,
        data_loaders = DepositModels.dataloaders,
        calculator_type = DepositIterativeCalculator,
        calc_type = DepositsCalculationType,
        adapter_types = DepositModels.adapter_types,
        scenario_data = scenario_df,
        portfolio = portfolio
    )

    training_manager = TrainingManager(spark, config.trainers, folder, model_db)   
    engine: ForecastEngine = ForecastEngine(spark, config, training_manager, overwrite_models=False)

    

    engine.run_all()
    return engine

In [10]:
from datetime import date, datetime

In [11]:
%%time

for month in range(7, 11):
    train_end = (date.today() + MonthEnd(-month)).to_pydatetime()
    print(train_end)
    engine = retrain_on_date(train_end)


INFO:core:missing models: []


2023-04-30 00:00:00


INFO:core:add_models_from_bytes
INFO:core:plan_close_201402_202304 - adapter <class 'core.models.plan_close.plan_close_model.PlanCloseModelAdapter'>
INFO:core:renewal_model_201401_202304 - adapter <class 'core.models.renewal.renewal_model.RenewalModelAdapter'>
INFO:core:maturity_structure_mass_r0s0_201401_202304 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S0ModelAdapter'>
INFO:core:maturity_structure_mass_r0s1_202001_202304 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S1ModelAdapter'>
INFO:core:maturity_structure_mass_r1s0_201401_202304 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S0ModelAdapter'>
INFO:core:maturity_structure_mass_r1s1_201401_202304 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S1ModelAdapter'>
INF

2023-03-31 00:00:00


INFO:core:add_models_from_bytes
INFO:core:plan_close_201402_202303 - adapter <class 'core.models.plan_close.plan_close_model.PlanCloseModelAdapter'>
INFO:core:renewal_model_201401_202303 - adapter <class 'core.models.renewal.renewal_model.RenewalModelAdapter'>
INFO:core:maturity_structure_mass_r0s0_201401_202303 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S0ModelAdapter'>
INFO:core:maturity_structure_mass_r0s1_202001_202303 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S1ModelAdapter'>
INFO:core:maturity_structure_mass_r1s0_201401_202303 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S0ModelAdapter'>
INFO:core:maturity_structure_mass_r1s1_201401_202303 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S1ModelAdapter'>
INF

2023-02-28 00:00:00


INFO:core:add_models_from_bytes
INFO:core:plan_close_201402_202302 - adapter <class 'core.models.plan_close.plan_close_model.PlanCloseModelAdapter'>
INFO:core:renewal_model_201401_202302 - adapter <class 'core.models.renewal.renewal_model.RenewalModelAdapter'>
INFO:core:maturity_structure_mass_r0s0_201401_202302 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S0ModelAdapter'>
INFO:core:maturity_structure_mass_r0s1_202001_202302 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S1ModelAdapter'>
INFO:core:maturity_structure_mass_r1s0_201401_202302 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S0ModelAdapter'>
INFO:core:maturity_structure_mass_r1s1_201401_202302 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S1ModelAdapter'>
INF

2023-01-31 00:00:00


INFO:core:add_models_from_bytes
INFO:core:plan_close_201402_202301 - adapter <class 'core.models.plan_close.plan_close_model.PlanCloseModelAdapter'>
INFO:core:renewal_model_201401_202301 - adapter <class 'core.models.renewal.renewal_model.RenewalModelAdapter'>
INFO:core:maturity_structure_mass_r0s0_201401_202301 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S0ModelAdapter'>
INFO:core:maturity_structure_mass_r0s1_202001_202301 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR0S1ModelAdapter'>
INFO:core:maturity_structure_mass_r1s0_201401_202301 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S0ModelAdapter'>
INFO:core:maturity_structure_mass_r1s1_201401_202301 - adapter <class 'core.models.newbusiness.maturity_structure.maturity_structure_mass_model.MaturityStructureMassR1S1ModelAdapter'>
INF

CPU times: user 2min 12s, sys: 19.7 s, total: 2min 32s
Wall time: 1min 5s


In [12]:
# вывод
portfolio_res = engine.calc_results['Deposits']['portfolio'] 
agg_res = engine.calc_results['Deposits']['agg_data']
maturity = engine.calc_results['Deposits']['maturity']
CurrentAccounts = engine.calc_results['CurrentAccounts']
SavingAccounts = engine.calc_results['SavingAccounts']
volumes = engine.calc_results['Volumes']

In [13]:
agg_res.groupby('report_dt').sum()

,replenishable_flg,subtraction_flg,month_maturity,target_maturity_days,start_balance,balance_gain,balance,newbusiness,contract_close,early_withdrawal,operations,interests,renewal,universal_weight_id
report_dt,,,,,,,,,,,,,,
2023-02-28,32,18,873,26532,2.950907e+12,1.210627e+11,3.071969e+12,7.677129e+11,-4.469379e+11,-8.080201e+10,-1.324373e+11,1.352700e+10,7.080551e+10,251
2023-03-31,32,18,873,26532,3.071969e+12,1.671942e+11,3.239163e+12,7.909402e+11,-4.013514e+11,-1.374526e+11,-1.012421e+11,1.630011e+10,1.071690e+11,251
2023-04-30,32,18,873,26532,3.239163e+12,1.838877e+11,3.423051e+12,5.345050e+11,-2.413509e+11,-4.816688e+10,-8.258664e+10,2.148710e+10,3.582332e+10,251


In [14]:
# для записи и чтения экселя
# import pip
# pip.main(['install', 'openpyxl'])

In [15]:
# import openpyxl

In [16]:
# with pd.ExcelWriter("august_res_v2.xlsx") as writer:
#     portfolio_res.to_excel(writer, sheet_name='portfolio_res', index=False)
#     agg_res.to_excel(writer, sheet_name='agg_res', index=False)
#     maturity.to_excel(writer, sheet_name='maturity', index=False)
#     CurrentAccounts.to_excel(writer, sheet_name='CurrentAccounts', index=False)
#     SavingAccounts.to_excel(writer, sheet_name='SavingAccounts', index=False)
#     volumes.to_excel(writer, sheet_name='volumes', index=False)
#     pd.DataFrame(scenario_data).to_excel(writer, sheet_name='scenario', index=False)